In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
from workadays import workdays as wd
from datetime import date

In [2]:
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)

In [3]:
hoje = dt.date(2023,8,3)

In [4]:
# PRECIFICANDO LTN

principal = 1000
vencimento = dt.date(2026,7,1)
i = 11.3980

dias = wd.networkdays(hoje,vencimento,252)/252

PU = truncate(principal/(1+i/100)**(dias),6)

print(PU)

731.169983


In [5]:
# PRECIFICANDO NTN-F

principal = 1000
cupom = 0.1
vencimento = dt.date(2033,1,1)
i = 11.8329

taxa_cupom = round((((1+cupom)**0.5)-1),8)

if hoje.month < 7:
    fluxo_1 = '01/07/' + str(hoje.year)
else:
    fluxo_1 = '01/01/'+ (str(hoje.year + 1))

fluxo_total = []
fluxo_total.append(fluxo_1)
fluxo_1 = datetime.strptime(fluxo_1,'%d/%m/%Y').date()

while(fluxo_1 < vencimento):
    fluxo_n = '01/' + (str(fluxo_1.month + 6)) + '/' + str(fluxo_1.year)
    if float(str(fluxo_1.month + 6)) <= 7:
        fluxo_n = '01/07/' + str(fluxo_1.year)
    else:
        fluxo_n = '01/01/'+ (str(fluxo_1.year + 1))
    fluxo_total.append(fluxo_n)
    fluxo_1 = datetime.strptime(fluxo_n,'%d/%m/%Y').date()

fluxo_pgto = []
for data in fluxo_total:
    data = datetime.strptime(data,'%d/%m/%Y').date()
    data = wd.workdays(wd.workdays(data,-1),+1)
    fluxo_pgto.append(data)

diaano_pgto = []
for data in fluxo_pgto:
    data = round((wd.networkdays(hoje,data,252))/252,16)
    diaano_pgto.append(data)
    
pg_cupom = []
for data in diaano_pgto:
    valor_cupom = round((taxa_cupom*principal)/(1+i/100)**data,8)
    pg_cupom.append(valor_cupom)
    
valor_principal = round(principal/(1+i/100)**(diaano_pgto[-1]),8)

val_cupom = round(pd.DataFrame(pg_cupom).sum(),8)

PU = truncate(val_cupom + valor_principal,6)

PU

911.562115